In [38]:
import pandas as pd
from tensorflow.keras.preprocessing import image

import tensorflow as tf

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet_v2 import ResNet152V2

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import activations
from tqdm import tqdm
import numpy as np

In [39]:
physical_devices = tf.config.list_physical_devices('GPU') 
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
cpus = tf.config.experimental.list_physical_devices(device_type='CPU')
print(gpus, cpus)

[] [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [42]:
#Unaugmented
df = pd.read_csv('HAM10000_metadata.csv')

In [43]:
#Unaugmented
NUM_VAL = 1000
NUM_TRN = 10015 - 1000
TOTAL = 10015

In [44]:
df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [58]:
df['localization'].unique()

array(['scalp', 'ear', 'face', 'back', 'trunk', 'chest',
       'upper extremity', 'abdomen', 'unknown', 'lower extremity',
       'genital', 'neck', 'hand', 'foot', 'acral'], dtype=object)

In [59]:
len(df)

10015

In [60]:
TOTAL

10015

In [48]:
# feature_vector = []

# for i in tqdm(range(0, 22242)):
#     sex = [1, 0, 0]
#     if df.iloc[i]['sex'] == 'male':
#         sex = [0, 1, 0]
#     elif df.iloc[i]['sex'] == 'female':
#         sex = [0, 0, 1]
        
#     age = df.iloc[i]['age'] / 80
#     loc = df.iloc[i]['localization']
    
#     feat = np.array(sex)

#     if loc == 'abdomen':
#         feat = np.concatenate((feat, tf.one_hot(0, 15)))# - 0.5))
#     elif loc == 'scalp':
#         feat = np.concatenate((feat, tf.one_hot(1, 15)))# - 0.5))
#     elif loc == 'lower extremity':
#         feat = np.concatenate((feat, tf.one_hot(2, 15)))# - 0.5))
#     elif loc == 'trunk':
#         feat = np.concatenate((feat, tf.one_hot(3, 15)))# - 0.5))
#     elif loc == 'upper extremity':
#         feat = np.concatenate((feat, tf.one_hot(4, 15)))# - 0.5))
#     elif loc == 'back':
#         feat = np.concatenate((feat,tf.one_hot(5, 15)))# - 0.5))
#     elif loc == 'neck':
#         feat = np.concatenate((feat,tf.one_hot(6, 15)))# - 0.5))
#     elif loc == 'face':
#         feat = np.concatenate((feat,tf.one_hot(7, 15)))# - 0.5))
#     elif loc == 'chest':
#         feat = np.concatenate((feat,tf.one_hot(8, 15)))# - 0.5))
#     elif loc == 'foot':
#         feat = np.concatenate((feat,tf.one_hot(9, 15)))# - 0.5))
#     elif loc == 'ear':
#         feat = np.concatenate((feat,tf.one_hot(10, 15)))# - 0.5))
#     elif loc == 'unknown':
#         feat = np.concatenate((feat,tf.one_hot(11, 15)))# - 0.5))
#     elif loc == 'hand':
#         feat = np.concatenate((feat,tf.one_hot(12, 15)))# - 0.5))
#     elif loc == 'acral':
#         feat = np.concatenate((feat,tf.one_hot(13, 15)))# - 0.5))
#     elif loc == 'genital':
#         feat = np.concatenate((feat,tf.one_hot(14, 15)))# - 0.5))

#     feature_vector.append(feat)

# feat_X = np.array(feature_vector)

# del feature_vector

In [53]:
#Unagumented
risk = []

for i in range(0, 10015):
    if df.iloc[i]['dx'] in ['bkl', 'df', 'vasc']:
        risk.append(0.0)
    elif df.iloc[i]['dx'] == 'nv':
        risk.append(1.0)
    elif df.iloc[i]['dx'] == 'akiec':
        risk.append(2.0)
    elif df.iloc[i]['dx'] in ['mel', 'bcc']:
        risk.append(3.0)
        
df['risk'] = risk

df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,risk
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,0.0
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,0.0
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,0.0
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,0.0
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,0.0


In [54]:
np.set_printoptions(threshold=np.inf)

In [55]:
from tensorflow.keras.callbacks import EarlyStopping

earlystop_callback = EarlyStopping(
  monitor='val_accuracy', min_delta=0.001,
  patience=3, restore_best_weights=True)

In [56]:
y_trn = np.array(df['risk'])[:(-1 * NUM_VAL)]
y_val = np.array(df['risk'])[(-1 * NUM_VAL):]

In [57]:
# VGG-16
preprocessed = []

VGG_load = VGG16(weights='imagenet', include_top=False)
model = models.Sequential()
model.add(VGG_load)
model.add(layers.Flatten())


for i in tqdm(range(0, TOTAL)):
    file = df.iloc[i]['image_id']
    feat = feat_X[i]
    
    img = image.load_img('./Data/HAM10000_images/' + file + '.jpg', target_size= (224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.vgg16.preprocess_input(img)
    preds = model.predict(img).reshape(25088)
    preprocessed.append(preds)

#     preprocessed.append(np.concatenate((preds, feat)))
'''mx = 0
for i in range(0, TOTAL):
    if mx < np.amax(preprocessed[i]):
        mx = np.amax(preprocessed[i])

preprocessed = list(preprocessed / mx)
'''
VGG_X = preprocessed

100%|██████████| 10015/10015 [31:54<00:00,  5.23it/s] 


In [61]:
VGG_X_val = np.array(VGG_X)

In [62]:
VGG_X_trn = np.array(VGG_X)[:(-1 * NUM_VAL)]

VGG_X_val = np.array(VGG_X)[(-1 * NUM_VAL):]

del VGG_X

MemoryError: Unable to allocate 958. MiB for an array with shape (10015, 25088) and data type float32

In [ ]:
new_y_trn = []
for ele in y_trn:
    new_y_trn.append(np.array(tf.one_hot(ele, 4)))
    
new_y_trn = np.array(new_y_trn)

new_y_trn.shape

In [ ]:
new_y_val = []
for ele in y_val:
    new_y_val.append(np.array(tf.one_hot(ele, 4)))
    
new_y_val = np.array(new_y_val)

new_y_val.shape

In [ ]:
new_VGG_trn = []

for i in tqdm(range(0, NUM_TRN)):
    new_VGG_trn.append(np.array(VGG_X_trn[i]))

new_VGG_trn = np.array(new_VGG_trn)

In [ ]:
new_VGG_val = []

for i in tqdm(range(0, NUM_VAL)):
    new_VGG_val.append(np.array(VGG_X_val[i]))

new_VGG_val = np.array(new_VGG_val)

In [ ]:
vgg_model = models.Sequential()
vgg_model.add(layers.Dense(8192, activation='relu'))
vgg_model.add(layers.Dense(4096, activation='relu'))
vgg_model.add(layers.Dense(2048, activation='relu'))
vgg_model.add(layers.Dense(1024, activation='relu'))
vgg_model.add(layers.Dense(512, activation='relu'))
vgg_model.add(layers.Dense(128, activation='relu'))
vgg_model.add(layers.Dense(4))

In [ ]:
vgg_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005),
              loss=tf.nn.softmax_cross_entropy_with_logits,
              metrics=['accuracy'])
vgg_model.fit(new_VGG_trn, new_y_trn, 
              epochs=30, 
              validation_data=(new_VGG_val, new_y_val), 
              callbacks=[earlystop_callback])

In [ ]:
#Save vgg model
vgg_model.save("VGG_Augmented.h5")

In [ ]:
predict_vgg_std = np.array(predict_y_vgg_val).std()
predict_vgg_mean = np.array(predict_y_vgg_val).mean()

print("vgg std: ", predict_vgg_std)
print("vgg mean: " , predict_vgg_mean)

In [63]:
#Load Vgg model
vgg_model = models.load_model('VGG_Augmented.h5', compile = False)

OSError: SavedModel file does not exist at: VGG_Augmented.h5\{saved_model.pbtxt|saved_model.pb}

In [64]:
del temp_vgg_val
del vgg_model
del VGG_X_trn
del VGG_X_val

NameError: name 'temp_vgg_val' is not defined

In [ ]:
# ResNet
preprocessed = []

resnet_load = ResNet152V2(weights='imagenet', include_top=False)
model = models.Sequential()
model.add(resnet_load)
model.add(layers.Flatten())


for i in tqdm(range(0, TOTAL)):
    file = df.iloc[i]['image_id']
    feat = feat_X[i]    
    
    img = image.load_img('./Data/HAM10000_images/' + file + '.jpg', target_size= (224,224))
    #img = image.load_img('./Unaug_Data/HAM10000_images/' + file + '.jpg', target_size= (224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.resnet_v2.preprocess_input(img)
    preds = model.predict(img).reshape(100352)
    preprocessed.append(preds)

#     preprocessed.append(np.concatenate((preds, feat)))



'''mx = 0
for i in range(0, TOTAL):
    if mx < np.amax(preprocessed[i]):
        mx = np.amax(preprocessed[i])

preprocessed = list(preprocessed / mx)
'''

RES_X = preprocessed

 42%|████▏     | 4232/10015 [1:28:08<2:38:09,  1.64s/it]

In [ ]:
#Weight Set up
class_weights = {3:4,  1:0.3, 2:25, 0:4}

sample_weights = []

for i in range(0, 10015):
    sample_weights.append(class_weights[df.iloc[i]['risk']])
    
df['weight'] = sample_weights
sample_weights = np.array(sample_weights)
sample_weights = sample_weights[:-1000]

In [ ]:
temp_res_trn.shape

In [ ]:
#RES_X_trn = np.array(RES_X)[:(-1 * NUM_VAL)]

RES_X_val = np.array(RES_X)[(-1 * NUM_VAL):]

del RES_X

In [ ]:
new_RES_trn = []

for i in tqdm(range(0, NUM_TRN)):
    new_RES_trn.append(np.array(RES_X_trn[i]))

new_RES_trn = np.array(new_RES_trn)

In [ ]:
new_RES_val = []

for i in tqdm(range(0, NUM_VAL)):
    new_RES_val.append(np.array(RES_X_val[i]))

new_RES_val = np.array(new_RES_val)

In [ ]:
res_model = models.Sequential()
#res_model.add(layers.Dense(8192, activation='relu'))
res_model.add(layers.Dense(4096, activation='relu'))
#res_model.add(layers.Dense(2048, activation='relu'))
res_model.add(layers.Dense(1024, activation='relu'))
#res_model.add(layers.Dense(512, activation='relu'))
res_model.add(layers.Dense(128, activation='relu'))
res_model.add(layers.Dense(4))

In [ ]:
res_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005),
              loss=tf.nn.softmax_cross_entropy_with_logits,
              metrics=['accuracy'])
res_model.fit(new_RES_trn, new_y_trn, 
              epochs=30, callbacks=[earlystop_callback],
              validation_data=(new_RES_val, new_y_val),
             class_weight = class_weights)

In [ ]:
predict_y_res_val = []
temp_res_val = res_model.predict(new_RES_val)
#temp_res_trn = res_model.predict(new_RES_trn)

#del new_RES_trn
#del new_RES_val

for i in tqdm(range(0, NUM_VAL)):
    pred = np.argmax(temp_res_val[i])
    predict_y_res_val.append(pred)

In [ ]:
predict_res_std = np.array(predict_y_res_val).std()
predict_res_mean = np.array(predict_y_res_val).mean()


print("res std: ", predict_res_std)
print("res mean: " , predict_res_mean)


In [ ]:
res_model = models.load_model("RES_Augmentated.h5", compile = False)

In [ ]:
# Inception
preprocessed = []

IV3_load = InceptionV3(weights='imagenet', include_top=False)
model = models.Sequential()
model.add(IV3_load)
model.add(layers.Flatten())

for i in tqdm(range(0, TOTAL)):
    file = df.iloc[i]['image_id']
    feat = feat_X[i]
    
    #img = image.load_img('./Data/HAM10000_images/' + file + '.jpg', target_size= (299,299))
    img = image.load_img('./Unaug_Data/HAM10000_images/' + file + '.jpg', target_size= (299,299))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    preds = model.predict(img).reshape(131072)

    preprocessed.append(np.concatenate((preds, feat)))


    
'''mx = 0
for i in range(0, TOTAL):
    if mx < np.amax(preprocessed[i]):
        mx = np.amax(preprocessed[i])

preprocessed = list(preprocessed / mx)
'''
IV3_X = preprocessed

In [19]:
IV3_X_trn = np.array(IV3_X)[:(-1 * NUM_VAL)]

IV3_X_val = np.array(IV3_X)[(-1 * NUM_VAL):]

del IV3_X

NameError: name 'IV3_X' is not defined

In [ ]:
new_IV3_trn = []

for i in tqdm(range(0, NUM_TRN)):
    new_IV3_trn.append(np.array(IV3_X_trn[i]))

new_IV3_trn = np.array(new_IV3_trn)

new_IV3_trn.shape

In [ ]:
new_IV3_val = []

for i in tqdm(range(0, NUM_VAL)):
    new_IV3_val.append(np.array(IV3_X_val[i]))

new_IV3_val = np.array(new_IV3_val)

In [ ]:
iv3_model = models.Sequential()
#iv3_model.add(layers.Dense(8192, activation='relu'))
iv3_model.add(layers.Dense(1024, activation='relu'))
#iv3_model.add(layers.Dense(2048, activation='relu'))
iv3_model.add(layers.Dense(256, activation='relu'))
#iv3_model.add(layers.Dense(512, activation='relu'))
#iv3_model.add(layers.Dense(128, activation='relu'))
iv3_model.add(layers.Dense(4))

In [ ]:
iv3_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005),
              loss=tf.nn.softmax_cross_entropy_with_logits,
              metrics=['accuracy'])
iv3_model.fit(new_IV3_trn, new_y_trn, 
              epochs=30, callbacks=[earlystop_callback],
              validation_data=(new_IV3_val, new_y_val))

In [ ]:
predict_y_iv3_val = []
temp_iv3_trn = iv3_model.predict(new_IV3_trn)
temp_iv3_val = iv3_model.predict(new_IV3_val)

del new_IV3_trn
del new_IV3_val

for i in tqdm(range(0, NUM_VAL)):
    pred = np.argmax(temp_iv3_val[i])
    predict_y_iv3_val.append(pred)

In [ ]:
"""max_iv3,"""
max_vgg, max_res = 0, 0


#max_iv3 = np.amax([np.amax(temp_iv3_trn), np.amax(temp_iv3_val)])
max_vgg = np.amax([np.amax(temp_vgg_trn), np.amax(temp_vgg_val)])
max_res = np.amax([np.amax(temp_res_trn), np.amax(temp_res_val)])

In [20]:
new_X_trn = []

for i in tqdm(range(0, NUM_TRN)):
    feat = np.concatenate(( #(temp_iv3_trn[i] / max_iv3), 
                            (temp_vgg_trn[i] / max_vgg), 
                            (temp_res_trn[i] / max_res) ))
    new_X_trn.append(feat)
    
new_X_trn = np.array(new_X_trn)
new_X_trn.shape

  0%|          | 0/24174 [00:00<?, ?it/s]


NameError: name 'temp_vgg_trn' is not defined

In [ ]:
new_X_val = []

for i in tqdm(range(0, NUM_VAL)):
    feat = np.concatenate(( #(temp_iv3_val[i]), 
                            (temp_vgg_val[i]/max_vgg), 
                            (temp_res_val[i]/max_res)))
    new_X_val.append(feat)
    
new_X_val = np.array(new_X_val)
new_X_val.shape

In [ ]:
final_model = models.Sequential()

#final_model.add(layers.Dense(64, activation='relu'))
#final_model.add(layers.Dense(32, activation='relu'))
#final_model.add(layers.Dense(16, activation='relu'))
#final_model.add(layers.Dense(8, activation='relu'))
final_model.add(layers.Dense(4))

In [ ]:
final_model.compile(optimizer='Adam',
              loss=tf.nn.softmax_cross_entropy_with_logits,
              metrics=['accuracy'])
final_model.fit(new_X_trn, new_y_trn, 
                epochs=20, callbacks=[earlystop_callback],
                validation_data=(new_X_val, new_y_val))

In [ ]:
predict_y_final_val = []
temp_final_val = final_model.predict(new_X_val)

for i in tqdm(range(0, NUM_VAL)):
    pred = np.argmax(temp_final_val[i])
    predict_y_final_val.append(pred)

In [ ]:
cm_final = tf.math.confusion_matrix(np.array(df['risk'])[(-1 * NUM_VAL):], np.array(predict_y_final_val))

cm_final = np.array(cm_final).astype('float32')

cm_final[0] = cm_final[0] / (1.0 * cm_final[0].sum())
cm_final[1] = cm_final[1] / (1.0 * cm_final[1].sum())
cm_final[2] = cm_final[2] / (1.0 * cm_final[2].sum())
cm_final[3] = cm_final[3] / (1.0 * cm_final[3].sum())

In [ ]:
sns.heatmap(cm_final, annot=True)